## Connect GoogleColab

In [4]:
# 如果出現了長條型的可輸入單元格 代表你曾經 授權過了 可以直接跳到下一格
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
··········
Error: Mountpoint . should be an existing directory.


In [5]:
# 指定Google Drive雲端硬盤的根目錄，名為drive
!mkdir -p drive
!google-drive-ocamlfuse drive

mkdir: cannot create directory ‘drive’: Transport endpoint is not connected
Error: Mountpoint drive should be an existing directory.


In [6]:
import os
os.chdir("drive")
if("Colab Notebooks" in os.listdir()):
  os.chdir("Colab Notebooks")
else:
  os.mkdir("Colab Notebooks")
  os.chdir("Colab Notebooks")

OSError: ignored

## 一、載入相關套件

In [0]:
# 資料處理套件
import cv2
import csv
import random
import time
import numpy as np
import pandas as pd
import matplotlib.image as mpimg # mpimg 用於讀取圖片
import matplotlib.pyplot as plt # plt 用於顯示圖片
import seaborn as sns

In [0]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 

In [0]:
# Keras深度學習模組套件
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras import utils as np_utils
from keras import backend as K
from keras import optimizers

In [0]:
# tensorflow深度學習模組套件
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

In [0]:
# 打印相關版本
print(pd.__version__)
print(tf.__version__)

In [0]:
# 查看通道位置
print(K.image_data_format())

In [0]:
!ls

## 二、讀取資料

In [0]:
# 讀取資料集標籤檔
Sample_label = pd.read_csv("AIMango/train.csv",encoding="utf8")

In [0]:
# 顯示資料集標籤檔
Sample_label.head()

In [0]:
# 串接圖片檔的路徑
Sample_pics_path = os.path.join("AIMango/C1-P1_Train")

In [0]:
# 讀取路徑中的圖片
train_mango_fnames = os.listdir(Sample_pics_path)

In [0]:
# 檢視是否有讀取到圖片
print(train_mango_fnames[0])
print(train_mango_fnames[1])
print(train_mango_fnames[2])

In [0]:
# 用於瀏覽標籤檔的概況
label_Survey = pd.read_csv("AIMango/dev.csv",encoding="utf8")

In [0]:
label_Survey.head()

In [0]:
sns.countplot(label_Survey['label'], hue = label_Survey["label"])

In [0]:
sector = label_Survey.groupby('label')
sector.size()

## 三、顯示芒果圖片

In [0]:
# 讀取圖檔
img = mpimg.imread("AIMango/C1-P1_Train/00007.jpg")
# 查看資料型態
type(img)

In [0]:
# 顯示圖片的比例
img.shape

In [0]:
# 把圖片的比例壓縮至800x800 
res = cv2.resize(img,(200,200),interpolation=cv2.INTER_LINEAR)

In [0]:
# 顯示壓縮後圖片的比例
res.shape

In [0]:
# 顯示原圖的芒果照片
plt.imshow(img)
plt.axis('off')
plt.show()

In [0]:
# 顯示壓縮過原圖的芒果照片
plt.imshow(res)
plt.axis('off')
plt.show()

## 四、製作標籤&資料集

In [0]:
csvfile = open('AIMango/train.csv')
reader = csv.reader(csvfile)

In [0]:
# 讀取csv標籤
labels = []
for line in reader:
    tmp = [line[0],line[1]]
    # print tmp
    labels.append(tmp)

csvfile.close() 

In [0]:
picnum = len(labels)
print("芒果圖片數量: ",picnum)

In [0]:
labels[8]

In [0]:
X = []
y = []

In [0]:
# 轉換圖片的標籤
for i in range(len(labels)):
    labels[i][1] = labels[i][1].replace("A","0")
    labels[i][1] = labels[i][1].replace("B","1")
    labels[i][1] = labels[i][1].replace("C","2")

In [0]:
# 隨機讀取圖片
a = 0
items= []

In [0]:
import random
for a in range(1,5601):
    items.append(a)

In [0]:
# 製作訓練用資料集及標籤
j=0
for i in random.sample(items,5600):
    img = cv2.imread("AIMango/C1-P1_Train/" + labels[i][0] )
    res = cv2.resize(img,(200,200),interpolation=cv2.INTER_LINEAR)
    res = img_to_array(res)
    X.append(res)    
    y.append(labels[i][1])
    j=j+1
    print(j)

In [0]:
y_label_org = y

In [0]:
img[0]

In [0]:
print(len(X))
print(len(y))

In [0]:
# 轉換至array的格式
X = np.array(X)
y = np.array(y)

In [0]:
# 轉換至float的格式
for i in range(len(X)):
    X[i] = X[i].astype('float32')

In [0]:
# 打映圖片訓練集的概況
# print(X[0])
print(type(X))
print(X.shape)

print(X[0].shape)
print(type(X[0]))

In [0]:
# 將標籤轉換至float格式
y = tf.strings.to_number(y, out_type=tf.float32)

In [0]:
# 打映圖片標籤的概況
print(y[0])
print(type(y[0]))

In [0]:
# 標籤進行one-hotencoding
y = np_utils.to_categorical(y, num_classes = 3)

In [0]:
y[0]

## 五、製作訓練資料集

In [0]:
# 分配訓練集及測試集比例
x_train = X[:5600]
y_train = y[:5600]
x_test = X[5600:]
y_test = y[5600:]

In [0]:
y_test

In [0]:
y_train_label = [0.,0.,0.]

for i in range(0,len(y_train)):
    y_train_label = y_train[i] + y_train_label

In [0]:
y_test_label = [0.,0.,0.]

for i in range(0,len(y_test)):
    y_test_label = y_train[i] + y_test_label

In [0]:
y_train_label

In [0]:
y_test_label

In [0]:
print(type(x_train))
print(len(x_train))
print(x_train.shape)
print(type(x_train[0]))

## 六、建立與訓練深度學習Model

In [0]:
# 建立深度學習CNN Model

model = tf.keras.Sequential()

model.add(layers.Conv2D(16,(3,3),     #16
                 strides=(1,1),
                 input_shape=(200, 200, 3),
#                 kernal_initializer='he_uniform',
                 padding='same',
                 activation='relu',
                 ))

model.add(layers.BatchNormalization())

model.add(layers.MaxPooling2D(pool_size=(2,2),strides=None))

model.add(layers.Conv2D(28,(3,3),
                 strides=(1,1),
#                 kernal_initializer='he_uniform',
                 padding='same',
                 activation='relu',
                 ))

model.add(layers.MaxPooling2D(pool_size=(2,2),strides=None))

model.add(layers.Conv2D(9,(3,3),
                 strides=(1,1),
#                 kernal_initializer='he_uniform',
                 padding='same',
                 activation='relu',
                 ))

model.add(layers.MaxPooling2D(pool_size=(2,2),strides=None))

#model.add(layers.Dropout(0.25))

model.add(layers.Flatten())

model.add(layers.Dense(64,activation='relu'))

model.add(layers.Dense(128,activation='relu'))

model.add(layers.Dropout(0.5))

model.add(layers.Dense(3,activation='softmax'))

model.summary()

history = model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [0]:
adam = optimizers.adam(lr=2)
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['acc'])

In [0]:
# zca_whitening 對輸入數據施加ZCA白化
# rotation_range 數據提升時圖片隨機轉動的角度
# width_shift_range 圖片寬度的某個比例，數據提升時圖片水平偏移的幅度
# shear_range 剪切強度（逆時針方向的剪切變換角度）
# zoom_range 隨機縮放的幅度
# horizontal_flip 進行隨機水平翻轉
# fill_mode ‘constant’，‘nearest’，‘reflect’或‘wrap’之一，當進行變換時超出邊界的點將根據本參數給定的方法進行處理

datagen = ImageDataGenerator(
    zca_whitening=False,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [0]:
# 導入圖像增強參數
datagen.fit(x_train)
x_train = x_train/255
x_test = x_test/255
print('rescale！done!')

In [0]:
# 設定超參數HyperParameters 
batch_size =  10
epochs = 120

In [0]:
# 檔名設定
file_name = str(epochs)+'_'+str(batch_size)

In [0]:
# 加入EarlyStopping以及Tensorboard等回調函數
CB = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
TB = keras.callbacks.TensorBoard(log_dir='./log'+"_"+file_name, histogram_freq=1)

In [0]:
history = model.fit(
    x = x_train , y = y_train,
    batch_size = batch_size,
    epochs = epochs,
    validation_split = 0.2,
    callbacks = [CB]
)

## 柒、繪製Model學習成效

In [0]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,1)
    plt.show()

In [0]:
plot_learning_curves(history)

## 捌、推測圖片

In [0]:
test_mango_dir = os.path.join("AIMango/C1-P1_Dev")
test_mango_fnames = os.listdir(test_mango_dir)

In [0]:
test_mango_fnames[0]

In [0]:
img_files = [os.path.join(test_mango_dir,f) for f in test_mango_fnames]
img_path = random.choice(img_files)

# 讀入待測試圖像並秀出
img = load_img(img_path, target_size=(200, 200))  # this is a PIL image
plt.title(img_path)
plt.grid(False)
plt.imshow(img)

In [0]:
labels = ['A','B',"C"]

In [0]:
# 將圖像轉成模型可分析格式(800x800x3, float32)
x = img_to_array(img)  # Numpy array with shape (800, 800, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 800, 800, 3)
x /= 255 # Rescale by 1/255

start = time.time() # 啟動計時器
result = model.predict(x) # 對輸入圖像進行推論(預測)
finish = time.time() # 結束計時器

pred = result.argmax(axis=1)[0]
pred_prob = result[0][pred]

print("Result = %f" %pred_prob) # 印出結果可能機率值(0.0 ~ 1.0)
print("Test time :%f second." %(finish-start)) # 印出推論時間

# 設定分類門檻值並印出推論結果
print("有 {:.2f}% 機率為{}".format(pred_prob * 100,labels[pred])) # 印出推論時間

## 玖、測試集預測準確度

In [0]:
T_csvfile = open("AIMango/dev.csv")
T_reader = csv.reader(T_csvfile)

T_labels = []
for line in T_reader:
  tmp = [line[0],line[1]]
  T_labels.append(tmp)

T_csvfile.close();

x = []
y = []

for i in range(len(T_labels)):
  T_labels[i][1] = T_labels[i][1].replace("A","0")
  T_labels[i][1] = T_labels[i][1].replace("B","1")
  T_labels[i][1] = T_labels[i][1].replace("C","2")

a=0
T_items = []
import random
for a in range(1,801):
  T_items.append(a)
j=0
for i in random.sample(T_items,800):
  img = cv2.imread("AIMango/C1-P1_Dev/" + T_labels[i][0] )
  res = cv2.resize(img,(200,200),interpolation=cv2.INTER_LINEAR)
  res = img_to_array(res)
  x.append(res)    
  y.append(T_labels[i][1])  
  j=j+1
  print(j)

x = np.array(x)
y = np.array(y)

for i in range(len(x)):
    x[i] = x[i].astype('float32')

y_label_org = y
y = tf.strings.to_number(y, out_type=tf.float32)
y = np_utils.to_categorical(y, num_classes = 3)

x_test = x
y_test = y

In [0]:
# 測試集標籤預測
y_pred = model.predict(x_test)

In [0]:
# 整體準確度
count = 0
for i in range(len(y_pred)):
    if(np.argmax(y_pred[i]) == np.argmax(y_test[i])): #argmax函数找到最大值的索引，即为其类别
        count += 1
score = count/len(y_pred)
print('正确率为:%.2f%s' % (score*100,'%'))

In [0]:
# 模型預測後的標籤
predict_label = np.argmax(y_pred,axis=1)
print(predict_label)
print(len(predict_label))

In [0]:
# 模型原標籤
true_label = y_label_org
true_label = np.array(true_label)
print(true_label)
print(len(true_label))

In [0]:
# 模型預測後的標籤
predictions = model.predict_classes(x_test)
print(predictions)
print(len(predictions))

In [0]:
pd.crosstab(true_label,predict_label,rownames=['實際值'],colnames=['預測值'])

In [0]:
# 儲存模型相關參數
model.save('model.h5')